# Discossional Chatbot using pre-trained model
We will use a model from the Hugging Face library.

# setup our environment

### Setup python environment
1. Download Anaconda through this link https://www.anaconda.com/download and install
2. Once Anaconda is installed, luanch Jupyter notebook and copy this command "!pip install transformers torch tkinter" to install the libraries
3. the required libraries are transformers, tourch and tkinter.

In [2]:
# Import the libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from transformers.utils import logging

## Download the selected pre-trained model and tokenize it

In [4]:
logging.get_logger("transformers").setLevel(logging.ERROR)
# Load the model and tokenizer
# model_name = "microsoft/DialoGPT-medium"
model_name = "microsoft/DialoGPT-large"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')
model = AutoModelForCausalLM.from_pretrained(model_name)

## Test run the model with limit of 5 transactions

In [6]:
# testing the pre_trained model by chatting 5 times with the chatbot
for step in range(5):
    # take user input
    text = input(">> You:")
    # encode the input and add end of string token
    input_val = tokenizer.encode(text + tokenizer.eos_token, return_tensors="pt")
    # concatenate new user input with chat history (if there is)
    bot_input_val = torch.cat([chat_history_val, input_val], dim=-1) if step > 0 else input_val
    # generate a bot response
    chat_history_val = model.generate(
        bot_input_val,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
    )
    #print the output
    output = tokenizer.decode(chat_history_val[:, bot_input_val.shape[-1]:][0], skip_special_tokens=True)
    print(f"Chatbot: {output}")

>> You: how are you


Chatbot: I'm good, you?


>> You: what is the weather today


Chatbot: Cloudy and raining.


>> You: oh is it heavy


Chatbot: It's raining.


>> You: oh how much rain


Chatbot: It's raining.


>> You: well its ok


Chatbot: It's raining again.


### Putting all together in a user interface using tkinter library

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import tkinter as tk
from tkinter import scrolledtext
from transformers.utils import logging

class ChatbotApp:
    def __init__(self, root):
        # create the user interface frame and add the components
        self.root = root
        self.root.title("Chatbot")

        self.chat_window = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=50, height=20)
        self.chat_window.grid(column=0, row=0, padx=10, pady=10, columnspan=2)
        self.chat_window.configure(state='disabled')

        self.entry_label = tk.Label(root, text="Ask your question:")
        self.entry_label.grid(column=0, row=1, padx=10, pady=10)

        self.entry_text = tk.Entry(root, width=40)
        self.entry_text.grid(column=0, row=2, padx=10, pady=10)

        self.ask_button = tk.Button(root, text="Ask", command=self.ask_question)
        self.ask_button.grid(column=1, row=2, padx=10, pady=10)

        # Load the model and tokenizer
        logging.get_logger("transformers").setLevel(logging.ERROR)
        model_name = "microsoft/DialoGPT-large"
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')
        self.model = AutoModelForCausalLM.from_pretrained(model_name)

        self.step = 0
        
    # Get chatbot response to questions
    def get_response(self, text):
        # encode the input and add end of string token
        input_val = self.tokenizer.encode(text + self.tokenizer.eos_token, return_tensors="pt")
        # concatenate new user input with chat history (if there is)
        bot_input_val = torch.cat([self.chat_history_val, input_val], dim=-1) if self.step > 0 else input_val
        # generate a bot response
        self.chat_history_val = self.model.generate(
            bot_input_val,
            max_length=1000,
            pad_token_id=self.tokenizer.eos_token_id,
        )
        # get the response to the output
        output = self.tokenizer.decode(self.chat_history_val[:, bot_input_val.shape[-1]:][0], skip_special_tokens=True)

        self.step +=1
        return f"{output}"

    def ask_question(self):
        question = self.entry_text.get()
        if question:
            self.chat_window.configure(state='normal')
            self.chat_window.insert(tk.END, "You: " + question + "\n")
            self.chat_window.configure(state='disabled')
            self.entry_text.delete(0, tk.END)
            
            response = self.get_response(question)
            self.chat_window.configure(state='normal')
            self.chat_window.insert(tk.END, "Bot: " + response + "\n\n")
            self.chat_window.configure(state='disabled')
            self.chat_window.yview(tk.END)

if __name__ == "__main__":
    root = tk.Tk()
    app = ChatbotApp(root)
    root.mainloop()
